In [16]:
# libraries
import os
import requests
import pandas as pd
import json
import re

DOE_LEAD_URL = "https://lead.openei.org/api/geographies"

In [2]:
fpl = ["0-100", "100-150", "150-200", "200-400", "400+"]

In [9]:

# Read in all possible census tracts
census_tracts = pd.read_csv("data/census_tracts_2010.csv")
# Get all unique states
states = census_tracts["state"].unique()

In [13]:
mi_tracts = census_tracts[census_tracts["state"] == "Michigan"]["geoid"].unique()
mi_geoids = [str(x) for x in mi_tracts]
geoids_str = ",".join(mi_geoids)
geoids_str

'26001000100,26001970100,26001970400,26001970500,26001970600,26001990000,26003000100,26003000200,26003000300,26003990000,26005030200,26005030300,26005030401,26005030402,26005030500,26005030600,26005030702,26005030703,26005030704,26005030800,26005030901,26005030902,26005031000,26005031100,26005031200,26005031300,26005031800,26005031900,26005032000,26005032100,26005032200,26005032401,26005032402,26005032600,26005990000,26007000100,26007000200,26007000300,26007000400,26007000500,26007000600,26007000700,26007000800,26007000900,26007990000,26009960100,26009960200,26009960300,26009960400,26009960500,26009960600,26009960700,26009990000,26011970100,26011970200,26011970300,26011970400,26011970500,26011990000,26011990100,26013000100,26013000200,26013990100,26015010100,26015010200,26015010300,26015010401,26015010402,26015010500,26015010600,26015010700,26015010800,26015011300,26015011400,26017280300,26017280400,26017280500,26017280600,26017280700,26017280800,26017280900,26017281000,26017281300,260

In [15]:
outdir_json = "outputs/MI_FPL/JSON"

for curr_fpl in fpl:
   # HTTP post request for energy burden across all MI census tracts for a specific AMI group
      resp = requests.post(
         DOE_LEAD_URL,
         json = {
            "type": "census-tracts", # Make sure to include type: "census-tracts" in post requests to get data by census tracts
            "geoids": geoids_str, # This should be a string of census tract geoids comma separated no whitespace
            "fpl": [curr_fpl] # Data will be aggregated across all AMI groups requested
         }
      )

      print(f'{curr_fpl} {resp.status_code}')
      # Get JSON data from HTTP response
      data = resp.json()
      # Serialize JSON data
      data_serialized = json.dumps(data, indent = 3)
      # Write JSON data out
      out_f = open(f'{outdir_json}/mi_fpl_{curr_fpl}.json', "w")
      out_f.write(data_serialized)
      out_f.close()

0-100 200
100-150 200
150-200 200
200-400 200
400+ 200


In [17]:
# Clean JSON data and structure as CSV
json_files = os.listdir(outdir_json)

outdir_csv = "outputs/MI_FPL/CSV"

for in_file in json_files:
   print(f'{outdir_json}/{in_file}')
   f = open(f'{outdir_json}/{in_file}')
   data = f.read()
   f.close()

   # curr_state = re.findall("^.*?(?=\_)", in_file)[0]
   curr_state = "MI"
   curr_fpl = re.findall("\d+-\d\d+|\d\d\d\+", in_file)[0]

   data = json.loads(data)
   total_items = data['total_items']
   items = data['items']

   df = pd.DataFrame.from_dict(items)
   df["state"] = curr_state
   df["fpl"] = curr_fpl
   outfilename = f'{in_file[:-5]}.csv'
   print(f'Writing {os.path.join(outdir_csv, outfilename)}')
   df.to_csv(os.path.join(outdir_csv, outfilename), index=False)

print("DONE")

outputs/MI_FPL/JSON/mi_fpl_150-200.json
Writing outputs/MI_FPL/CSV/mi_fpl_150-200.csv
outputs/MI_FPL/JSON/mi_fpl_100-150.json
Writing outputs/MI_FPL/CSV/mi_fpl_100-150.csv
outputs/MI_FPL/JSON/mi_fpl_200-400.json
Writing outputs/MI_FPL/CSV/mi_fpl_200-400.csv
outputs/MI_FPL/JSON/mi_fpl_400+.json
Writing outputs/MI_FPL/CSV/mi_fpl_400+.csv
outputs/MI_FPL/JSON/mi_fpl_0-100.json
Writing outputs/MI_FPL/CSV/mi_fpl_0-100.csv
DONE
